In [ ]:
!pip install transformers
!pip install transformers[sentencepiece]
!pip install datasets

Load banking77 dataset

In [6]:
from datasets import load_dataset

banking = load_dataset('banking77')
banking

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

Dataset banking77 downloaded and prepared to /root/.cache/huggingface/datasets/banking77/default/1.1.0/ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3080
    })
})

In [9]:
# Read CSV content
train = banking['train']
test = banking['test']

train.shape, test.shape

((10003, 2), (3080, 2))

In [16]:
import pandas as pd

train = pd.DataFrame(train)
test = pd.DataFrame(test)


In [21]:
train

,text,label
0,I am still waiting on my card?,11
1,What can I do if my card still hasn't arrived ...,11
2,I have been waiting over a week. Is the card s...,11
3,Can I track my card while it is in the process...,11
4,"How do I know if I will get my card, or if it ...",11
...,...,...
9998,You provide support in what countries?,24
9999,What countries are you supporting?,24
10000,What countries are getting support?,24
10001,Are cards available in the EU?,24


In [22]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Check tokenizer

In [24]:
train['text'].iloc[10]

'Why has my new card still not come?'

In [25]:
token = tokenizer.encode_plus(
    train['text'].iloc[10],
    max_length=256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='tf'
)

token

{'input_ids': <tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[ 101, 2009, 1144, 1139, 1207, 3621, 1253, 1136, 1435,  136,  102,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0, 

Creat arrays for input ids and attention masks for the BERT model

In [26]:
import numpy as np

X_input_ids = np.zeros((len(train), 256))
X_attn_masks = np.zeros((len(train), 256))

Transform Dataset

In [29]:
from tqdm.auto import tqdm

# generate tokenized training data
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        ) 
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [30]:
X_input_ids, X_attn_masks = generate_training_data(train, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [31]:
labels = np.zeros((len(train), 77))
labels.shape

(10003, 77)

In [32]:
# one-hot encoded target tensor
labels[np.arange(len(train)), train['label'].values] = 1 

labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
import tensorflow as tf

# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(77,), dtype=tf.float64, name=None))>

In [36]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [37]:
# converting to required format for tensorflow dataset
dataset = dataset.map(SentimentDatasetMapFunction) 

dataset.take(1) # one sample data

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(77,), dtype=tf.float64, name=None))>

In [38]:
# batch size, drop any left out tensor
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) 

In [39]:
# Split train set and validation set
p = 0.8
train_size = int((len(train)//16)*p)

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

Fit BertModel to the dataset

In [40]:
# bert base model with pretrained weights

from transformers import TFBertModel

model = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [41]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(77, activation='softmax', name='output_layer')(intermediate_layer)

banking77_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
banking77_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [42]:
# define optimizer, loss function and the accuracy matrix
optimzr = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

# compile the model
banking77_model.compile(optimizer=optimzr, loss=loss_func, metrics=[acc])

In [44]:
from tensorflow.keras.callbacks import EarlyStopping

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)

In [ ]:
hist = banking77_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[monitor],
)

Epoch 1/100
500/500 [==============================] - 456s 893ms/step - loss: 3.4376 - accuracy: 0.2770 - val_loss: 1.9264 - val_accuracy: 0.6920
Epoch 2/100
500/500 [==============================] - 451s 901ms/step - loss: 1.4707 - accuracy: 0.7489 - val_loss: 0.7719 - val_accuracy: 0.8650
Epoch 3/100
500/500 [==============================] - 451s 902ms/step - loss: 0.7250 - accuracy: 0.8677 - val_loss: 0.3795 - val_accuracy: 0.9265
Epoch 4/100
500/500 [==============================] - 451s 902ms/step - loss: 0.4296 - accuracy: 0.9169 - val_loss: 0.2480 - val_accuracy: 0.9515
Epoch 5/100
500/500 [==============================] - 451s 903ms/step - loss: 0.2814 - accuracy: 0.9427 - val_loss: 0.1484 - val_accuracy: 0.9710
Epoch 6/100
500/500 [==============================] - 451s 902ms/step - loss: 0.2025 - accuracy: 0.9579 - val_loss: 0.1051 - val_accuracy: 0.9805
Epoch 7/100
500/500 [==============================] - 451s 902ms/step - loss: 0.1561 - accuracy: 0.9665 - val_loss: 0

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def summarize_loss(history):
  history_dict = history.history
  loss = history_dict['loss']
  val_loss = history_dict['val_loss']

  epochs = range(1, len(loss) + 1)

  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.show()

def summarize_accuracy(history):
  history_dict = history.history
  acc = history_dict['accuracy']
  val_acc = history_dict['val_accuracy']

  epochs = range(1, len(acc) + 1)

  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()

  plt.show()

In [ ]:
summarize_loss(hist)

In [ ]:
summarize_accuracy(hist)

Save model

In [ ]:
import pickle

# save the model
filename = 'manking77_model.sav'
pickle.dump(banking77_model, open(filename, 'wb'))